In [1]:
import os
import re
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory

In [2]:
load_dotenv()

True

In [3]:
LANGCHAIN_TRACING_V2 = "true"
LANGSMITH_ENDPOINT = "https://api.smith.langchain.com"
LANGSMITH_PROJECT = "Information_Extractor"
LANGCHAIN_API_KEY = os.getenv("LANGSMITH_API_KEY")
OPENAI_API_KEY = os.getenv("GROQ_API")

In [ ]:
# Initialize the Groq LLM through LangChain
llm = ChatOpenAI(
    model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.0,
)

/var/folders/3c/n16ntvpx71gczgh_k1wjf9_c0000gp/T/ipykernel_21872/3100698544.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [5]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/var/folders/3c/n16ntvpx71gczgh_k1wjf9_c0000gp/T/ipykernel_21872/3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [6]:
application_info = {
    "name" : None,
    "email" : None,
    "skills" : None
}

In [7]:
def extract_application_info(text: str) -> str:
    name_match = re.search(r"(?:my name is|i am)\s+([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)", text, re.IGNORECASE) 
    email_match = re.search(r"\b[\w.-]+@[\w.-]+\.\w+\b", text)  
    skills_match = re.search(r"(?:skills are|i know|i can use)\s+(.+)", text, re.IGNORECASE)
    
    response = []
    
    if name_match:
        application_info['name'] = name_match
        response.append("Name Saved")
        
    if email_match:
        application_info["email"] = email_match.group(0)
        response.append("✅ Email saved.")
    
    if skills_match:
        application_info["skills"] = skills_match.group(1).strip()
        response.append("✅ Skills saved.")
        
    if not any([name_match, email_match, skills_match]):
        return "❓ I couldn't extract any info. Could you please provide your name, email, or skills?"

    return " ".join(response) + " Let me check what else I need."

In [8]:
def check_application_goal(_: str) -> str:
    if all(application_info.values()):
        return f"✅ You're ready! Name: {application_info['name']}, Email: {application_info['email']}, Skills: {application_info['skills']}."
    else:
        missing = [k for k, v in application_info.items() if not v]
        return f"⏳ Still need: {', '.join(missing)}. Please ask the user to provide this."

In [9]:
tools = [
    Tool(
        name="extract_application_info",
        func=extract_application_info,
        description="Use this to extract name, email, and skills from the user's message."
    ),
    Tool(
        name="check_application_goal",
        func=check_application_goal,
        description="Check if name, email, and skills are provided. If not, tell the user what is missing."
    )
]

In [10]:
SYSTEM_PROMPT = """You are a helpful job application assistant. 
Your goal is to collect the user's name, email, and skills. 
Use the tools provided to extract this information and check whether all required data is collected.
Once everything is collected, inform the user that the application info is complete and stop.
"""

In [11]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    memory = memory,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={"system_message": SYSTEM_PROMPT}
)

/var/folders/3c/n16ntvpx71gczgh_k1wjf9_c0000gp/T/ipykernel_21872/897628637.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [12]:
print("📝 Hi! I'm your job application assistant. Please tell me your name, email, and skills.")

📝 Hi! I'm your job application assistant. Please tell me your name, email, and skills.


In [13]:
while True:
    
    user_input = input("You: ")
    
    if user_input.lower() in ["exit", "quit"]:
        print("Bye! Good Luck")
        break
    
    response = agent.invoke({"input": user_input})
    print("Bot: ", response["output"])
    
    # if Goal achieved, stop
    if "you're ready" in response["output"].lower():
        print("Application Info Completed.")




> Entering new AgentExecutor chain...
```
{
    "action": "extract_application_info",
    "action_input": "my name yash, email is yash@gm.com and skills are python."
}
```
Observation: ✅ Email saved. ✅ Skills saved. Let me check what else I need.
Thought:```
{
    "action": "Final Answer",
    "action_input": "I have your email as yash@gm.com and skills as python. But I'm still missing your name."
}
```

> Finished chain.
Bot:  I have your email as yash@gm.com and skills as python. But I'm still missing your name.


> Entering new AgentExecutor chain...
```
{
    "action": "extract_application_info",
    "action_input": "my name is yash, email is yash@gm.com and skills are python"
}
```
Observation: Name Saved ✅ Email saved. ✅ Skills saved. Let me check what else I need.
Thought:```
{
    "action": "Final Answer",
    "action_input": "Application info is complete. Your name is yash, email is yash@gm.com and skills are python."
}
```

> Finished chain.
Bot:  Application info is comple